# Module 5 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

## Solving Normal Form Games

Add whatever additional imports you require here. Stick with the standard libraries and those required by the class. The import gives you access to these functions: https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html (Copy this link) Which, among other things, will permit you to display HTML as the result of evaluated code (see HTML() or display_html()).

In [1]:
from IPython.core.display import *
from typing import List, Tuple, Dict, Callable
from itertools import combinations

In the lecture we talked about the Prisoner's Dilemma game, shown here in Normal Form:

Player 1 / Player 2  | Defect | Cooperate
------------- | ------------- | -------------
Defect  | -5, -5 | -1, -10
Cooperate  | -10, -1 | -2, -2

where the payoff to Player 1 is the left number and the payoff to Player 2 is the right number. We can represent each payoff cell as a Tuple: `(-5, -5)`, for example. We can represent each row as a List of Tuples: `[(-5, -5), (-1, -10)]` would be the first row and the entire table as a List of Lists:

In [2]:
prisoners_dilemma = [
 [( -5, -5), (-1,-10)],
 [(-10, -1), (-2, -2)]]

prisoners_dilemma

[[(-5, -5), (-1, -10)], [(-10, -1), (-2, -2)]]

in which case the strategies are represented by indices into the List of Lists. For example, `(Defect, Cooperate)` for the above game becomes `prisoners_dilemma[ 0][ 1]` and returns the payoff `(-1, -10)` because 0 is the first row of the table ("Defect" for Player 1) and 1 is the 2nd column of the row ("Cooperate" for Player 2).

For this assignment, you are going write a function that uses Successive Elimination of Dominated Strategies (SEDS) to find the **pure strategy** Nash Equilibrium of a Normal Form Game. The function is called `solve_game`:

```python
def solve_game( game: List[List[Tuple]], weak=False) -> Tuple:
    pass # returns strategy indices of Nash equilibrium or None.
```

and it takes two parameters: the game, in a format that we described earlier and an optional boolean flag that controls whether the algorithm considers only **strongly dominated strategies** (the default will be false) or whether it should consider **weakly dominated strategies** as well.

It should work with game matrices of any size and it will return the **strategy indices** of the Nash Equilibrium. If there is no **pure strategy** equilibrium that can be found using SEDS, return `None`.


<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
Do not return the payoff. That's not useful. Return the strategy indices, any other output is incorrect.
</div>

As before, you must provide your implementation in the space below, one Markdown cell for documentation and one Code cell for implementation, one function and assertations per Codecell.


---

**player1**<br>
The `player1` is a helper a function for `solve_game`. It attemps to find strongly/weekly dominated strategy and drop it.

Parameters:
* **game** is the game we want find and remove the dominated strategy from.
* **ind_matrix** keeps track of which indices have been remove and which have not.
* **weak** indicate whether we should consider weakly dominated strategy.

retuns:<br>
If the dominated strategy is found then it is dropped and `game` is returned.

In [3]:
def player1(game, indx_matrix, weak = False):
    player_1_comb = list(combinations([*range(0,len(game), 1)],2))
    
    for row1,row2 in player_1_comb:
        if weak == False:
            check_row1 = all(game[row1][col][0] > game[row2][col][0] for col in range(len(game[0])))
            check_row2 = all(game[row2][col][0] > game[row1][col][0] for col in range(len(game[0])))
        else:
            # are values are equal ?
            check_equal_col = [game[row1][col][0] == game[row2][col][0] for col in range(len(game[0]))]
            check_equal_col = True if check_equal_col.count(True) > 0 and check_equal_col.count(True) < len(check_equal_col) else False    

            # make sure atleast one or more values are >
            check_greater_col1 = all(game[row1][col][0] >= game[row2][col][0] for col in range(len(game[0])))
            check_greater_col2 = all(game[row2][col][0] >= game[row1][col][0] for col in range(len(game[0])))
            
            check_row1 = check_greater_col1 & check_equal_col   # all valuea are >=
            check_row2 = check_greater_col2 & check_equal_col   # all valuea are >=
            
        if check_row1 == True:   # drop row2 from game and indx_matrix
            game.pop(row2), indx_matrix.pop(row2)
            break
        elif check_row2 == True: # drop row1 from game and indx_matrix
            game.pop(row1), indx_matrix.pop(row1)
            break
    return game

**player2**<br>
The `player2` is a helper a function for `solve_game`. It attemps to find strongly/weekly dominated strategy and drop it.

Parameters:
* **game** is the game we want find and remove the dominated strategy from.
* **ind_matrix** keeps track of which indices have been remove and which have not.
* **weak** indicate whether we should consider weakly dominated strategy.

retuns:<br>
If the dominated strategy is found then it is dropped and `game` is returned.

In [4]:
def player2(game, indx_matrix, weak = False):
    player_2_comb = list(combinations([*range(0,len(game[0]), 1)],2))

    for col1,col2 in player_2_comb:
        if weak == False:
            check_col1 = all(game[row][col1][1] > game[row][col2][1] for row in range(len(game)))
            check_col2 = all(game[row][col2][1] > game[row][col1][1] for row in range(len(game)))
        else:
            # make sure not all values are equal
            check_equal_col = [game[row][col1][1] == game[row][col2][1] for row in range(len(game))]
            check_equal_col = True if check_equal_col.count(True) > 0 and check_equal_col.count(True) < len(check_equal_col) else False    
            
            # make sure atleast one or more values are >
            check_greater_col1 = all(game[row][col1][1] >= game[row][col2][1] for row in range(len(game)))
            check_greater_col2 = all(game[row][col2][1] >= game[row][col1][1] for row in range(len(game)))
            
            check_col1 = check_greater_col1 & check_equal_col   # all valuea are != and are >=
            check_col2 = check_greater_col2 & check_equal_col   # all valuea are != and are >=
            
        if check_col1 == True: # drop col2 from game and indx_matrix
            [row.pop(col2) for row in game], [row.pop(col2) for row in indx_matrix]
            break
        elif check_col2 == True: # drop col1 from game and indx_matrix
            [row.pop(col1) for row in game], [row.pop(col1) for row in indx_matrix]
            break
    return game


---

In order to test your function you must describe three (3) test cases, each of which is a 3x3 two player game. You must indicate the solution.

**solve_game1**<br>
`solve game` uses Successive Elimination of Dominated Strategies (SEDS) to find the pure strategy Nash Equilibrium of a Normal Form Game. It can controls whether the algorithm considers only strongly dominated strategies or whether it should consider weakly dominated strategies as well. It works with game matrices of any size.


Parameters:
* **game** is the game we want to apply SEDS on.
* **weak** indicate whether it should consider weakly dominated strategy.

retuns:<br>
it will return the `strategy indices` of the Nash Equilibrium. If there is no pure strategy equilibrium that can be found using SEDS then it return `None`

In [5]:
def solve_game(game: List[List[Tuple]], weak:bool=False) -> Tuple:
    original_game = game.copy()
    indx_matrix = [[(row, col) for col in range(len(game[0]))]for row in range(len(game))]         

    while len(game)!= 1:
        prev = game.copy()
        game = player1(game.copy(), indx_matrix, weak)
        game = player2(game.copy(), indx_matrix, weak)

        if game == prev:
            return None # No change in game during this iteration (after both players turn)

    return indx_matrix[0][0]

### Test Game 1. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Strongly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 5,5 | 7,6 | 7,7 |
|1  | 6,7 | 9,9 | 14,7 |
|2  | 7,7 | 7,14 | 12,13|

**Solution:** (strategy indices)<br>
`(1,1)`

In [6]:
test_game_1 = [[(5,5), (7,6), (7,7)],
               [(6, 7), (9,9), (14, 7)],
               [(7, 7), (7, 14), (12, 13)]]

solution = solve_game(test_game_1)

In [7]:
assert solution == (1,1) # insert your solution from above.

### Test Game 2. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Weakly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 5,7 | 7,7 | 14,3 |
|1  | 5,5 | 7,6 | 11,7 |
|2  | 7,10 | 7,14 | 12,13|

**Solution:** (strategy indices)<br>
`(0,1)`

In [8]:
test_game_2 = [[(5, 7), (7,7), (14, 3)],
               [(5, 5), (7,6), (11,7)],
               [(7, 10), (7, 14), (12, 13)]]

strong_solution = solve_game( test_game_2)
weak_solution = solve_game( test_game_2, weak=True)

In [9]:
assert strong_solution == None
assert weak_solution == (0,1) # insert your solution from above.

### Test Game 3. Create a 3x3 two player game

**that cannot be solved using the Successive Elimintation of Dominated Strategies at all**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 5,5 | 5,6 | 25,7 |
|1  | 5,7 | 7,7 | 14,7 |
|2  | 7,25 | 7,14 | 12,13 |

**Solution:** None

In [10]:
test_game_3 =[[(5,5), (5,6), (25,7)],
               [(5, 7), (7,7), (14, 7)],
               [(7, 25), (7, 14), (12, 13)]]

strong_solution = solve_game(test_game_3)
weak_solution = solve_game(test_game_3, weak=True)

In [11]:
assert strong_solution == None
assert weak_solution == None

## Before You Submit...

1. Did you provide output exactly as requested? **Don't forget to fill out the Markdown tables with your games**.
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.